In [1]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

#import functions/Classes for sparkml

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

# import functions/Classes for metrics
from pyspark.ml.evaluation import RegressionEvaluator


# Examples


## Task 1 - Create a model


In [ ]:
#Create SparkSession
#Ignore any warnings by SparkSession command

spark = SparkSession.builder.appName("Model Persistence").getOrCreate()

Load the dataset into the spark dataframe


In [ ]:
# using the spark.read.csv function we load the data into a dataframe.
# the header = True mentions that there is a header row in out csv file
# the inferSchema = True, tells spark to automatically find out the data types of the columns.

# Load mpg dataset
mpg_data = spark.read.csv("mpg.csv", header=True, inferSchema=True)


Print the schema of the dataset


In [ ]:
mpg_data.printSchema()

Show top 5 rows from the dataset


In [ ]:
mpg_data.show(5)

We ask the VectorAssembler to group a bunch of inputCols as single column named "features"


In [ ]:
# Prepare feature vector
assembler = VectorAssembler(inputCols=["Cylinders", "Engine Disp", "Horsepower", "Weight", "Accelerate", "Year"], outputCol="features")
mpg_transformed_data = assembler.transform(mpg_data)


Display the assembled "features" and the label column "MPG"


In [ ]:
mpg_transformed_data.select("features","MPG").show()

We split the data set in the ratio of 70:30. 70% training data, 30% testing data.


In [ ]:
# Split data into training and testing sets
(training_data, testing_data) = mpg_transformed_data.randomSplit([0.7, 0.3])


Create a LR model and train the model using the pipeline on training data set


In [ ]:
# Train linear regression model
# Ignore any warnings
lr = LinearRegression(labelCol="MPG", featuresCol="features")
pipeline = Pipeline(stages=[lr])
model = pipeline.fit(training_data)


## Task 2 - Save the model


Create a folder where the model will to be saved


In [ ]:
!mkdir model_storage

In [ ]:
# Persist the model to the path "./model_stoarage/"

model.write().overwrite().save("./model_storage/")

#The overwrite method is used to overwrite the model if it already exists,
#and the save method is used to specify the path where the model should be saved.



## Task 3 - Load the model


In [ ]:
from pyspark.ml.pipeline import PipelineModel

# Load persisted model
loaded_model = PipelineModel.load("./model_storage/")

## Task 4 - Predict using the loaded model


In [ ]:
# Make predictions on test data
predictions = loaded_model.transform(testing_data)
#In the above example, we use the load method of the PipelineModel object to load the persisted model from disk. We can then use this loaded model to make predictions on new data using the transform method.


Your model is now trained. We use the testing data to make predictions.


In [ ]:
# Make predictions on testing data
predictions = model.transform(testing_data)

In [ ]:
predictions.select("prediction").show(5)

Stop Spark Session


In [ ]:
spark.stop()

<!--
|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-05-04|0.1|Ramesh Sannareddy|Initial Version Created|
-->
